# Web Scraping

<img src="https://www.hepsiemlak.com/emlak-yasam/wp-content/uploads/2018/04/ev-kirasi-neye-gore-artar.jpg">

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
  

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_column', 100)

In [3]:
base_url = 'https://www.emlakjet.com'
filter_url = '/kiralik-konut/istanbul/?min_fiyat=9000'

In [4]:

page = requests.get(base_url + filter_url)

In [5]:
page.content

b'<!DOCTYPE html><html lang="tr"><head><link rel="shortcut icon" type="image/png" href="/static/img/favicon-green.png"/><style>\n    /* roboto-regular - latin_latin-ext */\n    @font-face {\n      font-family: \'Roboto\';\n      font-style: normal;\n      font-display: swap;\n      font-weight: 400;\n      src: url(\'/static/fonts/roboto-regular.eot\'); /* IE9 Compat Modes */\n      src: local(\'Roboto\'), local(\'Roboto-Regular\'),\n        url(\'/static/fonts/roboto-regular.eot?#iefix\') format(\'embedded-opentype\'), /* IE6-IE8 */\n        url(\'/static/fonts/roboto-regular.woff2\') format(\'woff2\'), /* Super Modern Browsers */\n        url(\'/static/fonts/roboto-regular.woff\') format(\'woff\'), /* Modern Browsers */\n        url(\'/static/fonts/roboto-regular.ttf\') format(\'truetype\'), /* Safari, Android, iOS */\n        url(\'/static/fonts/roboto-regular.svg#Roboto\') format(\'svg\'); /* Legacy iOS */\n    }\n    /* roboto-500 - latin_latin-ext */\n    @font-face {\n      font

In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
paginations = soup.find('ul', {'class': '_49yNIg'}).find_all('li')[-2].text
paginations = int(paginations)
paginations

50

In [8]:
links = []

for i in range(1, paginations+1):
    page = requests.get(base_url + f'/kiralik-konut/istanbul/{i}/?min_fiyat=9000')
    soup = BeautifulSoup(page.content, 'html.parser')
    list = soup.find_all('a', {'class': '_3qUI9q'})

    for link in list:
        links.append(link.get('href')) 

In [9]:
len(links)

1500

In [19]:
page = requests.get(base_url + '/ilan/sancaktepe-meclis-de-hastane-ve-metroya-cok-yakin-23500-15789618/')
soup = BeautifulSoup(page.content, 'html.parser')

list = []
dict = {}

price_div = soup.find('div', {'class': '_2TxNQv'})
price = int(price_div.text.replace('TL', '').replace('.', '').replace(',', '').strip())
dict.update({'Fiyat': price})

location = soup.find('div', {'class': '_3VQ1JB'}).find('p').text
parts = location.split(' - ')
dict.update({'Şehir': parts[0]})
dict.update({'İlçe': parts[1]})
dict.update({'Mahalle': parts[2]})



features = soup.find_all('div', {'class': '_35T4WV'})
for feature in features:    
    divs = feature.find_all('div', class_='_1bVOdb')
    dict.update({divs[0].text: divs[1].text})


list.append(dict)
list

'İstanbul - Sancaktepe - Meclis Mahallesi'

In [20]:
list = []
for link in links:
    page = requests.get(base_url + link)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    dict = {}

    price_div = soup.find('div', {'class': '_2TxNQv'})

    if price_div is not None:
        price = price_div.text.replace('TL', '').replace('.', '').replace(',', '').strip()
        price = price.split('TL')[0] + 'TL'
        dict.update({'Fiyat': price})

    location = soup.find('div', {'class': '_3VQ1JB'})
    if location is not None:
        location = location.select('p')[0].text
        parts = location.split(' - ')
        dict.update({'Şehir': parts[0]})
        dict.update({'İlçe': parts[1]})
        dict.update({'Mahalle': parts[2]})


    features = soup.find_all('div', {'class': '_35T4WV'})
    if features is not None:
        for feature in features:    
            divs = feature.find_all('div', class_='_1bVOdb')
            dict.update({divs[0].text: divs[1].text})


    list.append(dict)

In [21]:
list

[{'Fiyat': '25000TL',
  'Şehir': 'İstanbul',
  'İlçe': 'Bakırköy',
  'Mahalle': 'Osmaniye Mahallesi',
  'İlan Numarası': '15794285',
  'İlan Güncelleme Tarihi': '30 Ağustos 2024',
  'Kategorisi': 'Kiralık',
  'Net Metrekare': '90 m²',
  'Oda Sayısı': '2+1',
  'Bulunduğu Kat': 'Yüksek Giriş',
  'Isıtma Tipi': 'Merkezi (Pay Ölçer)',
  'Depozito': '25000 TL',
  'Banyo Sayısı': '1',
  'İlan Oluşturma Tarihi': '30 Ağustos 2024',
  'Türü': 'Konut',
  'Tipi': 'Daire',
  'Brüt Metrekare': '100 m²',
  'Binanın Yaşı': '21 Ve Üzeri',
  'Binanın Kat Sayısı': '5',
  'Kullanım Durumu': 'Boş',
  'Site İçerisinde': 'Evet',
  'Fiyat Durumu': 'Genel Fiyat'},
 {'Fiyat': '14000TL',
  'Şehir': 'İstanbul',
  'İlçe': 'Küçükçekmece',
  'Mahalle': 'Kanarya Mahallesi',
  'İlan Numarası': '15792263',
  'İlan Güncelleme Tarihi': '29 Ağustos 2024',
  'Kategorisi': 'Kiralık',
  'Net Metrekare': '145 m²',
  'Oda Sayısı': '3+1',
  'Bulunduğu Kat': '6.Kat',
  'Isıtma Tipi': 'Kombi Doğalgaz',
  'Yapı Durumu': 'İkinci E

In [22]:
df = pd.DataFrame(list)
df = df.dropna(axis=1, how='all')
df.columns

Index(['Fiyat', 'Şehir', 'İlçe', 'Mahalle', 'İlan Numarası',
       'İlan Güncelleme Tarihi', 'Kategorisi', 'Net Metrekare', 'Oda Sayısı',
       'Bulunduğu Kat', 'Isıtma Tipi', 'Depozito', 'Banyo Sayısı',
       'İlan Oluşturma Tarihi', 'Türü', 'Tipi', 'Brüt Metrekare',
       'Binanın Yaşı', 'Binanın Kat Sayısı', 'Kullanım Durumu',
       'Site İçerisinde', 'Fiyat Durumu', 'Yapı Durumu', 'Eşya Durumu',
       'Balkon Durumu', 'Yapı Tipi', 'WC Sayısı', 'Takas', 'Banyo Metrekare',
       'Salon Metrekare', 'WC Metrekare', 'Görüntülü Gezilebilir mi?', 'Aidat',
       'Balkon Tipi', 'Balkon Sayısı', 'Balkon Metrekare', 'Ä°lan NumarasÄ±',
       'Ä°lan GÃ¼ncelleme Tarihi', 'Oda SayÄ±sÄ±', 'BulunduÄŸu Kat',
       'IsÄ±tma Tipi', 'YapÄ± Durumu', 'EÅŸya Durumu', 'Site Ä°Ã§erisinde',
       'Banyo SayÄ±sÄ±', 'Ä°lan OluÅŸturma Tarihi', 'TÃ¼rÃ¼',
       'BrÃ¼t Metrekare', 'BinanÄ±n YaÅŸÄ±', 'BinanÄ±n Kat SayÄ±sÄ±',
       'KullanÄ±m Durumu', 'YapÄ± Tipi', 'WC SayÄ±sÄ±',
       'GÃ¶rÃ¼ntÃ¼lÃ¼ G

In [23]:
drop_columns = ['Ä°lan NumarasÄ±',
'Ä°lan GÃ¼ncelleme Tarihi', 'Oda SayÄ±sÄ±', 'BulunduÄŸu Kat',
'IsÄ±tma Tipi', 'YapÄ± Durumu', 'EÅŸya Durumu', 'Site Ä°Ã§erisinde',
'Banyo SayÄ±sÄ±', 'Ä°lan OluÅŸturma Tarihi', 'TÃ¼rÃ¼',
'BrÃ¼t Metrekare', 'BinanÄ±n YaÅŸÄ±', 'BinanÄ±n Kat SayÄ±sÄ±',
'KullanÄ±m Durumu', 'YapÄ± Tipi', 'WC SayÄ±sÄ±',
'GÃ¶rÃ¼ntÃ¼lÃ¼ Gezilebilir mi?', 'Balkon SayÄ±sÄ±']
df = df.drop(drop_columns, axis=1)

In [24]:
df.sample(10)

,Fiyat,Şehir,İlçe,Mahalle,İlan Numarası,İlan Güncelleme Tarihi,Kategorisi,Net Metrekare,Oda Sayısı,Bulunduğu Kat,Isıtma Tipi,Depozito,Banyo Sayısı,İlan Oluşturma Tarihi,Türü,Tipi,Brüt Metrekare,Binanın Yaşı,Binanın Kat Sayısı,Kullanım Durumu,Site İçerisinde,Fiyat Durumu,Yapı Durumu,Eşya Durumu,Balkon Durumu,Yapı Tipi,WC Sayısı,Takas,Banyo Metrekare,Salon Metrekare,WC Metrekare,Görüntülü Gezilebilir mi?,Aidat,Balkon Tipi,Balkon Sayısı,Balkon Metrekare
857,20000TL,İstanbul,Beylikdüzü,Adnan Kahveci Mahallesi,15781084,30 Ağustos 2024,Kiralık,115 m²,1+1,Yüksek Giriş,Kombi Doğalgaz,NaN,1,28 Ağustos 2024,Konut,Daire,130 m²,16-20,13,Boş,Evet,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,25000TL,İstanbul,Sarıyer,Tarabya Mahallesi,15793149,29 Ağustos 2024,Kiralık,80 m²,1+1,Yüksek Giriş,Kombi Doğalgaz,25000 TL,1,29 Ağustos 2024,Konut,Daire,85 m²,21 Ve Üzeri,4,Boş,Hayır,Genel Fiyat,İkinci El,Boş,Var,Betonarme,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
472,33000TL,İstanbul,Tuzla,Evliya Çelebi Mahallesi,15780915,28 Ağustos 2024,Kiralık,130 m²,3.5+1,4.Kat,Kombi Doğalgaz,NaN,2,28 Ağustos 2024,Konut,Daire,150 m²,1,5,Boş,Hayır,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1067,16000TL,İstanbul,Sultangazi,Cebeci Mahallesi,15767292,25 Ağustos 2024,Kiralık,90 m²,2+1,Yüksek Giriş,Doğalgaz Sobalı,16000 TL,1,25 Ağustos 2024,Konut,Daire,100 m²,16-20,5,Boş,Hayır,Genel Fiyat,NaN,NaN,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 TL,NaN,1,NaN
822,45000TL,İstanbul,Gaziosmanpaşa,Karadeniz Mahallesi,15786584,28 Ağustos 2024,Kiralık,120 m²,3+1,6.Kat,Merkezi (Pay Ölçer),NaN,2,28 Ağustos 2024,Konut,Residence,185 m²,5-10,25,Boş,Evet,Genel Fiyat,NaN,Eşyalı,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,18000TL,İstanbul,Esenler,Kazım Karabekir Mahallesi,15789780,30 Ağustos 2024,Kiralık,100 m²,2+1,4.Kat,Kombi Doğalgaz,18000 TL,1,29 Ağustos 2024,Konut,Daire,105 m²,16-20,5,Boş,Hayır,Genel Fiyat,İkinci El,Boş,Var,Betonarme,2,NaN,5 m²,25 m²,3 m²,NaN,100 TL,Kapalı Balkon,1,4 m²
170,32000TL,İstanbul,Ümraniye,Elmalıkent Mahallesi,15792023,29 Ağustos 2024,Kiralık,70 m²,2+1,3.Kat,Merkezi (Pay Ölçer),NaN,2,29 Ağustos 2024,Konut,Daire,95 m²,0 (Yeni),10,Boş,Evet,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,20000TL,İstanbul,Silivri,Selimpaşa Mahallesi,15793299,29 Ağustos 2024,Kiralık,150 m²,3+1,Çatı Katı,Kombi Doğalgaz,20000 TL,1,29 Ağustos 2024,Konut,Daire,160 m²,11-15,4,Boş,Hayır,Genel Fiyat,İkinci El,Boş,Var,NaN,1,NaN,NaN,NaN,NaN,Hayır,NaN,NaN,NaN,NaN
1476,94990TL,İstanbul,Eyüpsultan,Mimar Sinan Mahallesi,15755307,29 Ağustos 2024,Kiralık,90 m²,2+1,Düz Giriş (Zemin),Merkezi Doğalgaz,NaN,1,23 Ağustos 2024,Konut,Daire,108 m²,1,3,Kiracı Oturuyor,Evet,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1106,16000TL,İstanbul,Üsküdar,Bulgurlu Mahallesi,15766601,28 Ağustos 2024,Kiralık,35 m²,Stüdyo,1.Kat,Sobalı,NaN,1,25 Ağustos 2024,Konut,Daire,40 m²,16-20,4,Boş,Hayır,Genel Fiyat,İkinci El,NaN,NaN,Betonarme,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df.columns

Index(['Fiyat', 'Şehir', 'İlçe', 'Mahalle', 'İlan Numarası',
       'İlan Güncelleme Tarihi', 'Kategorisi', 'Net Metrekare', 'Oda Sayısı',
       'Bulunduğu Kat', 'Isıtma Tipi', 'Depozito', 'Banyo Sayısı',
       'İlan Oluşturma Tarihi', 'Türü', 'Tipi', 'Brüt Metrekare',
       'Binanın Yaşı', 'Binanın Kat Sayısı', 'Kullanım Durumu',
       'Site İçerisinde', 'Fiyat Durumu', 'Yapı Durumu', 'Eşya Durumu',
       'Balkon Durumu', 'Yapı Tipi', 'WC Sayısı', 'Takas', 'Banyo Metrekare',
       'Salon Metrekare', 'WC Metrekare', 'Görüntülü Gezilebilir mi?', 'Aidat',
       'Balkon Tipi', 'Balkon Sayısı', 'Balkon Metrekare'],
      dtype='object')

In [26]:
column_translation = {
    'Fiyat': 'Price',
    'Şehir': 'City',
    'İlçe': 'District',
    'Mahalle': 'Neighborhood',
    'İlan Numarası': 'Listing Number',
    'İlan Güncelleme Tarihi': 'Listing Update Date',
    'Kategorisi': 'Category',
    'Net Metrekare': 'Net Square Meters',
    'Oda Sayısı': 'Number of Rooms',
    'Bulunduğu Kat': 'Floor',
    'Isıtma Tipi': 'Heating Type',
    'Site İçerisinde': 'Within Site',
    'Fiyat Durumu': 'Price Status',
    'İlan Oluşturma Tarihi': 'Listing Creation Date',
    'Türü': 'Type',
    'Tipi': 'Kind',
    'Brüt Metrekare': 'Gross Square Meters',
    'Binanın Yaşı': 'Building Age',
    'Binanın Kat Sayısı': 'Number of Building Floors',
    'Kullanım Durumu': 'Usage Status',
    'Banyo Sayısı': 'Number of Bathrooms',
    'Yapı Durumu': 'Structure Status',
    'Eşya Durumu': 'Furnishing Status',
    'Depozito': 'Deposit',
    'Takas': 'Swap',
    'Banyo Metrekare': 'Bathroom Square Meters',
    'Salon Metrekare': 'Living Room Square Meters',
    'WC Metrekare': 'WC Square Meters',
    'Görüntülü Gezilebilir mi?': 'Virtual Tour Available?',
    'Yapı Tipi': 'Building Type',
    'Aidat': 'Maintenance Fee',
    'Balkon Durumu': 'Balcony Status',
    'WC Sayısı': 'Number of WCs',
    'Balkon Tipi': 'Balcony Type',
    'Balkon Sayısı': 'Number of Balconies',
    'Balkon Metrekare': 'Balcony Square Meters'
}


df.rename(columns=column_translation, inplace=True)

In [27]:
df.sample(10)

,Price,City,District,Neighborhood,Listing Number,Listing Update Date,Category,Net Square Meters,Number of Rooms,Floor,Heating Type,Deposit,Number of Bathrooms,Listing Creation Date,Type,Kind,Gross Square Meters,Building Age,Number of Building Floors,Usage Status,Within Site,Price Status,Structure Status,Furnishing Status,Balcony Status,Building Type,Number of WCs,Swap,Bathroom Square Meters,Living Room Square Meters,WC Square Meters,Virtual Tour Available?,Maintenance Fee,Balcony Type,Number of Balconies,Balcony Square Meters
223,18000TL,İstanbul,Sultangazi,Cebeci Mahallesi,15789827,29 Ağustos 2024,Kiralık,95 m²,2+1,1.Kat,Kombi Doğalgaz,18000 TL,1,29 Ağustos 2024,Konut,Daire,100 m²,21 Ve Üzeri,4,Boş,Hayır,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 TL,NaN,NaN,NaN
1013,20000TL,İstanbul,Şişli,Eskişehir Mahallesi,15769414,26 Ağustos 2024,Kiralık,55 m²,1+1,5.Kat,Kombi Doğalgaz,NaN,1,26 Ağustos 2024,Konut,Daire,60 m²,11-15,5,Boş,Hayır,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1271,12000TL,İstanbul,Esenyurt,Bağlarçeşme Mahallesi,15762004,24 Ağustos 2024,Kiralık,80 m²,2+1,2.Kat,Kombi Doğalgaz,12000 TL,1,24 Ağustos 2024,Konut,Daire,90 m²,5-10,5,Boş,Hayır,Genel Fiyat,İkinci El,Boş,Var,NaN,NaN,Yok,NaN,25 m²,NaN,Evet,50 TL,NaN,1,NaN
1295,34000TL,İstanbul,Küçükçekmece,Cumhuriyet Mahallesi,15761365,24 Ağustos 2024,Kiralık,90 m²,2+1,5.Kat,Merkezi (Pay Ölçer),NaN,1,24 Ağustos 2024,Konut,Daire,100 m²,5-10,11,Boş,Evet,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
926,30000TL,İstanbul,Kadıköy,Acıbadem Mahallesi,15772934,26 Ağustos 2024,Kiralık,105 m²,3+1,Yüksek Giriş,Merkezi Doğalgaz,NaN,1,26 Ağustos 2024,Konut,Daire,110 m²,21 Ve Üzeri,5,Boş,Hayır,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,40000TL,İstanbul,Beylikdüzü,Marmara Mahallesi,15774269,27 Ağustos 2024,Kiralık,90 m²,2+1,6.Kat,Merkezi (Pay Ölçer),NaN,2,27 Ağustos 2024,Konut,Daire,130 m²,4,10,Boş,Evet,Genel Fiyat,NaN,Eşyalı,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409,18500TL,İstanbul,Bahçelievler,Fevzi Çakmak Mahallesi,15783262,30 Ağustos 2024,Kiralık,55 m²,1+1,1.Kat,Kombi Doğalgaz,NaN,1,28 Ağustos 2024,Konut,Daire,65 m²,0 (Yeni),4,Boş,Hayır,Genel Fiyat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,45000TL,İstanbul,Maltepe,Altıntepe Mahallesi,15797331,30 Ağustos 2024,Kiralık,100 m²,2+1,4.Kat,Kombi Doğalgaz,45000 TL,2,30 Ağustos 2024,Konut,Daire,110 m²,5-10,4,Boş,Hayır,Genel Fiyat,NaN,NaN,Var,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350 TL,NaN,NaN,NaN
785,30500TL,İstanbul,Kartal,Karlıktepe Mahallesi,15791192,29 Ağustos 2024,Kiralık,100 m²,3+1,1.Kat,Kombi Doğalgaz,30500 TL,1,29 Ağustos 2024,Konut,Daire,120 m²,16-20,4,Boş,Hayır,Genel Fiyat,İkinci El,Boş,Var,Betonarme,2,Var,NaN,NaN,NaN,Hayır,100 TL,Kapalı Balkon,1,NaN
1398,28000TL,İstanbul,Küçükçekmece,Kartaltepe Mahallesi,15758309,23 Ağustos 2024,Kiralık,78 m²,1+1,9.Kat,Merkezi (Pay Ölçer),NaN,1,23 Ağustos 2024,Konut,Residence,90 m²,4,15,Boş,Evet,Genel Fiyat,NaN,Boş,NaN,NaN,NaN,Yok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df.shape

(1500, 36)

In [29]:
df.to_csv('emlakjet.csv', index=False)